In [16]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [17]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [18]:
ratings = pd.read_csv('/Users/adithyaramanch/Documents/python/Project/archive/ratings.csv')
movies = pd.read_csv('/Users/adithyaramanch/Documents/python/Project/archive/movies.csv')

In [19]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [20]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [21]:
movies.shape

(10329, 3)

In [22]:
ratings.shape

(105339, 4)

In [23]:
movies.describe()

,movieId
count,10329.000000
mean,31924.282893
std,37734.741149
min,1.000000
25%,3240.000000
50%,7088.000000
75%,59900.000000
max,149532.000000


In [24]:
ratings.describe()

,userId,movieId,rating,timestamp
count,105339.000000,105339.000000,105339.000000,1.053390e+05
mean,364.924539,13381.312477,3.516850,1.130424e+09
std,197.486905,26170.456869,1.044872,1.802660e+08
min,1.000000,1.000000,0.500000,8.285650e+08
25%,192.000000,1073.000000,3.000000,9.711008e+08
50%,383.000000,2497.000000,3.500000,1.115154e+09
75%,557.000000,5991.000000,4.000000,1.275496e+09
max,668.000000,149532.000000,5.000000,1.452405e+09


In [25]:
movies.isna().sum().sum()

0

In [26]:
ratings.isna().sum().sum()

0

In [27]:
# Number of users
print('The ratings dataset has', ratings['userId'].nunique(), 'unique users')
# Number of movies
print('The ratings dataset has', ratings['movieId'].nunique(), 'unique movies')
# Number of ratings
print('The ratings dataset has', ratings['rating'].nunique(), 'unique ratings')
# List of unique ratings
print('The unique ratings are', sorted(ratings['rating'].unique()))

The ratings dataset has 668 unique users
The ratings dataset has 10325 unique movies
The ratings dataset has 10 unique ratings
The unique ratings are [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]


In [28]:
#list of all genres

genres = []
for genre in movies.genres:
    x = genre.split('|')
    #print(x)
    for i in x:
        if(i not in genres):
            genres.append(str(i))
            
genres

['Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Fantasy',
 'Romance',
 'Drama',
 'Action',
 'Crime',
 'Thriller',
 'Horror',
 'Mystery',
 'Sci-Fi',
 'IMAX',
 'War',
 'Musical',
 'Documentary',
 'Western',
 'Film-Noir',
 '(no genres listed)']

In [29]:
df_ = pd.merge(ratings,movies, how='left',on='movieId')
df_.head()

,userId,movieId,rating,timestamp,title,genres
0,1,16,4.0,1217897793,Casino (1995),Crime|Drama
1,1,24,1.5,1217895807,Powder (1995),Drama|Sci-Fi
2,1,32,4.0,1217896246,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,4.0,1217896556,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,4.0,1217896523,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [30]:
df_1 = df_.groupby(['title'])[['rating']].mean()
high_rated = df_1.nlargest(20,'rating')
high_rated.head()

,rating
title,
21 Up (1977),5.0
50 Children: The Rescue Mission of Mr. And Mrs. Kraus (2013),5.0
Air (2015),5.0
Andrei Rublev (Andrey Rublyov) (1969),5.0
Animal Farm (1954),5.0


In [31]:
#Objective 1: Popularity Based Recommendation
a = str(input('Enter Genre ((Use First Letter Capital)):'))
b = int(input('Minimum Number of Thresholds:'))
c = int(input('Number Recommendations:'))

df = movies[movies['genres'].str.contains(a)]

df1 = pd.merge(ratings,df, how='left',on='movieId')
df1 = df1.dropna()
df1.head()



trend=pd.DataFrame(df1.groupby('title')['rating'].mean())
trend['total number of ratings'] = pd.DataFrame(df1.groupby('title')['rating'].count()) 
trend.head()

trend = trend.sort_values(by=['rating'], ascending=False)
rslt_df = trend.loc[trend['total number of ratings'] >= b]
rslt_df.head(c)


#df2 = df1.groupby(['title'])[['rating']].mean()
#df2.head()

#df3 = df1.groupby('title')[['rating']].count()
#df3.head()

#df4 = pd.merge(df2,df3, how='left',on='movieId')
#df4.head()


Enter Genre ((Use First Letter Capital)):Animation
Minimum Number of Thresholds:100
Number Recommendations:3


,rating,total number of ratings
title,,
"Incredibles, The (2004)",3.926471,102
Toy Story 2 (1999),3.913462,104
Toy Story (1995),3.907328,232


In [32]:
# content-based recommender system

In [34]:
df2 = pd.merge(ratings,movies, how='left',on='movieId')
df2.head()

tv = TfidfVectorizer()
tfidf_matrix=tv.fit_transform(movies['genres'])

movie_user = df2.pivot_table(index='userId',columns='title',values='rating')
movie_user.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 (1979),...,[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),a/k/a Tommy Chong (2005),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
#  We can recommend the user a movie by calculating the cosine similarity between the entered movie and others.

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [36]:
indices=pd.Series(movies.index,index=movies['title'])
titles=movies['title']

def recommendations(title,d):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:d+1]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [37]:
title = str(input('Movie Title:'))
d = int(input('Number of Recommendations'))

recommendations(title,d)

Movie Title:Jumanji (1995)
Number of Recommendations5


55             Indian in the Cupboard, The (1995)
112             NeverEnding Story III, The (1994)
817               Escape to Witch Mountain (1975)
1604    Darby O'Gill and the Little People (1959)
1652                          Return to Oz (1985)
Name: title, dtype: object